In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

# 기타 세팅
# 1. 맷플롯립 한글 폰트 표기
fm.findSystemFonts()
font_location = '/usr/share/fonts/NanumFont/NanumBarunGothic.ttf'
font_name = fm.FontProperties(fname=font_location).get_name()
plt.rc('font', family=font_name)

# 열 전부 보기
pd.set_option('display.max_columns', 100)
pd.options.display.float_format = '{:.5f}'.format
# !jupyter trust 250509_live_work_analysis.ipynb

In [ ]:
# shut-down for memory
def exit():
    os._exit(00)

# 메모리 확인
def memory():
    %load_ext memory_profiler
    %memit

In [ ]:
# exit()
# memory()

In [ ]:
station = pd.read_csv('import_data/TB_KTS_STTN/202401/TB_KTS_STTN_20240102.csv').rename(columns={'정산사코드':'정산사ID'})
transport_df = pd.read_csv('import_data/TB_KTS_DWTCD_METROPOLITAN/202401/TB_KTS_DWTCD_METROPOLITAN_20240102.csv')
route_df = pd.read_csv('import_data/TB_KTS_ROUTE/202401/TB_KTS_ROUTE_20240102.csv')
routestation_df = pd.read_csv('import_data/TB_KTS_ROUTESTTN/202401/TB_KTS_ROUTESTTN_20240102.csv')

In [ ]:
# 1. 빌딩데이터로 대상지역 기준 정류장 집계

In [ ]:
building = pd.read_csv('upload/building.txt', delimiter = '|')
daejang = building[(building.road_name_address.str.contains('판교대장로')) & ((building.title_purpose_name =='단독주택') |(building.title_purpose_name =='공동주택'))]
daejang_center = daejang[['longitude', 'latitude']].mean()
daejang_center

In [ ]:
from shapely.geometry import Point
daejang_center['geometry'] = Point(daejang_center['longitude'], daejang_center['latitude'])
daejang_center = gpd.GeoDataFrame(daejang_center.to_frame().T, geometry = 'geometry')


In [ ]:
daejang_center.plot()

In [ ]:
# 500m 버퍼 만들고 그 안에 정류장 정보 집계

station_gdf = gpd.GeoDataFrame(station, geometry = gpd.points_from_xy(station['정류장GPSX좌표'], station['정류장GPSY좌표']), crs = 'EPSG:4326')

# 위경도 미터법 좌표계로 전환
daejang_center= daejang_center.set_crs(epsg=4326)
station_gdf= station_gdf.set_crs(epsg=4326)
daejang_center = daejang_center.to_crs(epsg=5179)
station_gdf = station_gdf.to_crs(epsg=5179)

# 500m 버퍼생성
buffer = daejang_center.buffer(700)



In [ ]:
# buffer.to_file('buffer.geojson')

In [ ]:
daejang_center

In [ ]:
# 시각화
ax = station_gdf.plot()
daejang_center.plot(ax=ax, color='red')
buffer.plot(ax=ax, color='gray', alpha=0.4)
ax.set_xlim(960000,963000)
ax.set_ylim(1928000,1931500)

In [ ]:
# 정류장 집계
buffer = gpd.GeoDataFrame(buffer, geometry =0, crs = 'EPSG:5179')
stations_selected = buffer.sjoin(station_gdf, how='left', predicate='intersects')
print(len(stations_selected))
stations_selected

In [ ]:
# 정류장ID 리스트로 집계
daejang_stations = stations_selected.정류장ID.to_list()
print(len(daejang_stations))
print(len(set(daejang_stations)))

In [ ]:
# 2. 목적 통행 집계

In [ ]:
# 수단통행 -> 목적 통행 예시
# transport_df[(transport_df['가상카드번호'] == '100626138965') & (transport_df['트랜잭션ID'] == 1)]
transport_df[(transport_df['가상카드번호'] == 'zy3hZeIBq+/lka8Jo2GL0JeRKV6ZY/OiKwNLvnmrU+4=')]
# transport_df

In [ ]:
agg_dict = {**{x : 'first' for x in ['정산사ID', '이용자유형코드(시스템)']},
            **{x : 'sum' for x in ['이용거리', '탑승시간']},
           **{x: 'first' for x in ['정산사승차정류장ID', '승차일시']},
           **{x: 'last' for x in ['정산사하차정류장ID', '하차일시']}}
agg_dict['환승건수'] = 'max'
# multi-lines
multi_lines = {}
multi_lines['승차정산지역코드'] = ('정산지역코드', 'first')
multi_lines['하차정산지역코드'] = ('정산지역코드', 'last')
multi_lines['승차교통수단구분'] = ('교통수단구분', 'first')
multi_lines['하차교통수단구분'] = ('교통수단구분', 'last')
multi_lines['승차노선ID'] = ('정산사노선ID', 'first')
multi_lines['하차노선ID'] = ('정산사노선ID', 'last')

print(agg_dict)
multi_lines

In [ ]:
# 혼재된 정산지역코드 하나의 데이터타입으로 정리
transport_df['교통수단구분'] = transport_df['교통수단코드'].apply(lambda x: 'T' if (199<x & x<300) else 'B')
transport_df.정산지역코드 = transport_df.정산지역코드.astype(str)

In [ ]:
# 1. 목적 통행으로 먼저 집계해야함: 그렇지 않으면 목적 통행 중 최초 수단 통행과 최종 수단 통행만 집계되어 정보가 부분적으로 집계될 것

main_trip = transport_df.groupby(['가상카드번호', '트랜잭션ID']).agg(agg_dict).reset_index()
support_trip = transport_df.groupby(['가상카드번호', '트랜잭션ID']).agg(**multi_lines).reset_index()
linked_trip = pd.concat([main_trip, support_trip.iloc[:, 2:]], axis=1)
linked_trip

In [ ]:
# 하차 정보 Null값 drop
print(len(linked_trip)) # 13124125
linked_trip.dropna(subset='정산사하차정류장ID', inplace=True)
print(len(linked_trip)) # 12891611 ( -232514, -1.8%)

In [ ]:
## 2.1. 목적 통행 불러오기

In [ ]:
# 목적 통행본 저장 & 다시 불러오기\
# linked_trip.to_csv('linked_trip_240102.csv')
linked_trip = pd.read_csv('linked_trip_240102.csv')
linked_trip.head()

In [ ]:
# 3. 정류장 위치 정보 결합

In [ ]:
# 조인할 것들로만 추리기
station_core = station[['지역코드', '교통수단구분', '정류장ID', '정류장GPSY좌표', '정류장GPSX좌표']]
station_core.head(3)

In [ ]:
# 승차 정류장과 하차 정류장 위치를 따로 붙이기
# 데이터타입 체크
linked_trip.승차정산지역코드 = linked_trip.승차정산지역코드.astype(str)
linked_trip.하차정산지역코드 = linked_trip.하차정산지역코드.astype(str)

In [ ]:
# 2-1. 위치 정보 결합( 승차정류장ID 기준 )
linked_station = linked_trip.merge(station_core, how='left', left_on = ['승차정산지역코드', '승차교통수단구분', '정산사승차정류장ID'], right_on = ['지역코드', '교통수단구분', '정류장ID'],indicator = True)
linked_station._merge.value_counts() # 결합건수/결합률: 12854835, 99.4%

In [ ]:
# 결합 확률 확인
round((linked_station['_merge'] == 'both').sum()/len(linked_station)*100,2)

In [ ]:
linked_station = linked_station.drop(columns = ['지역코드', '교통수단구분', '정류장ID','_merge']).rename(columns ={
    '정류장GPSY좌표': '승차정류장Y좌표', '정류장GPSX좌표' :'승차정류장X좌표'})

In [ ]:
# 2-2. 위치 정보 결합( 하차정류장ID 기준 )
linked_station =linked_station.merge(station_core, how='left', left_on = ['하차정산지역코드', '하차교통수단구분', '정산사하차정류장ID'], right_on = ['지역코드', '교통수단구분', '정류장ID'],indicator = True)
linked_station._merge.value_counts() # 결합건수/결합률: 1282844, 99.5%

In [ ]:
round((linked_station['_merge'] == 'both').sum()/len(linked_station)*100,2)

In [ ]:
linked_station = linked_station.drop(columns = ['지역코드', '교통수단구분', '정류장ID','_merge', 'Unnamed: 0']).rename(columns ={
    '정류장GPSY좌표': '하차정류장Y좌표', '정류장GPSX좌표' :'하차정류장X좌표'})
linked_station.head(3)

In [ ]:
# 4. 대장동 인근 정류장 집계

In [ ]:
# 대중교통 이용자 중 대장동 정류장 승하차 인원 집계
linked_daejang = linked_station[linked_station.정산사승차정류장ID.isin(daejang_stations)|linked_station.정산사하차정류장ID.isin(daejang_stations)]

# 대장동 기준으로 출발하는 인원, 통계 확인
# dtype 변경

linked_daejang['승차일시'] = pd.to_datetime(
    linked_daejang['승차일시'].astype(int).astype(str),
    format='%Y%m%d%H%M%S')
linked_daejang['하차일시'] = pd.to_datetime(
    linked_daejang['하차일시'].astype(int).astype(str),
    format='%Y%m%d%H%M%S')

board_daejang = linked_daejang[linked_daejang.정산사승차정류장ID.isin(daejang_stations)]
alight_daejang = linked_daejang[linked_daejang.정산사하차정류장ID.isin(daejang_stations)]
len(linked_daejang)

In [ ]:
# 승차인 건과 하차인 건 구분 개수
print('총 건수: ', len(linked_daejang))
linked_daejang.dropna(inplace=True)
print('Null값 드랍 후 건수: ', len(linked_daejang))
print('승차 총 건수: ',len(board_daejang))
print('하차 총 건수: ',len(alight_daejang))
print('내부 통행 건수: ',len(linked_daejang[linked_daejang.정산사승차정류장ID.isin(daejang_stations)&linked_daejang.정산사하차정류장ID.isin(daejang_stations)]))

In [ ]:
# 통행 건수 확인
daejang_count = linked_daejang.groupby('가상카드번호').size().to_frame().reset_index()
daejang_count[0].value_counts()
# daejang_count[0].mean()

In [ ]:
# 왕복인 건들만 필터
oneway_trip_daejang = linked_daejang[linked_daejang['가상카드번호'].map(linked_daejang.groupby('가상카드번호').size()==1)]
round_trip_daejang = linked_daejang[linked_daejang['가상카드번호'].map(linked_daejang.groupby('가상카드번호').size()==2)]
round_trip_daejang.head()

In [ ]:
# 4.1. 통행 1건인 경우 분석

In [ ]:
len(oneway_trip_daejang)

In [ ]:
# 기존 목적 테이블과 결합 전체 이동패턴에서도 1번인지 대상지 집계하여 1개 된건지 확인
oneway_tripper = list(oneway_trip_daejang.가상카드번호.unique())
entire_oneway = linked_trip[linked_trip.가상카드번호.isin(oneway_tripper)]
entire_oneway.groupby('가상카드번호').size().value_counts()

In [ ]:
station[station['정류장ID'] == 4179179]

In [ ]:
entire_oneway[entire_oneway['가상카드번호'].map(entire_oneway.groupby('가상카드번호').size()==2)].sort_values('가상카드번호')

In [ ]:
## 4.1. 평균이동시간, 평균이동거리

In [ ]:
board_daejang.head()

In [ ]:
# 평균 이동시간, 평균이동거리
board_mean_triptime = board_daejang.탑승시간.mean()
board_mean_tripdistance = board_daejang.이용거리.mean()
alight_mean_triptime = alight_daejang.탑승시간.mean()
alight_mean_tripdistance = alight_daejang.이용거리.mean()
print("대장동 승차 기준")
print("    평균 이동시간: ", round(board_mean_triptime,2), '초')
print("    평균 이동거리: ", round(board_mean_tripdistance,2), 'm')
print("대장동 하차 기준")
print("    평균 이동시간: ", round(alight_mean_triptime,2), '초')
print("    평균 이동거리: ", round(alight_mean_tripdistance,2), 'm')

In [ ]:
## 4.2. 30분 단위 승하차 횟수

In [ ]:
- 15일 병합 테스트 - 1주일치 보는 것 최대
- 반출 신청 -> 기반으로 사후 관련 서류 최종으로 드릴 것 소정의 기프티콘

In [ ]:
# 승차 시간 확인 - 30분 단위로
linked_daejang[linked_daejang.정산사승차정류장ID.isin(daejang_stations)].승차일시.nunique()

In [ ]:
# 30 분 단위 집계 - 승차 기준
board_plot_daejang = board_daejang.set_index('승차일시')
board_plot_daejang = board_plot_daejang.resample('30T').size().rename('탑승건수').reset_index()
board_plot_daejang

In [ ]:
board_daejang.plot(kind='bar')

In [ ]:
# 30 분 단위 집계 - 하차 기준
alight_daejang = linked_daejang[linked_daejang['정산사하차정류장ID'].isin(daejang_stations)]
alight_daejang = alight_daejang.set_index('하차일시')
alight_daejang = alight_daejang.resample('30T').size().rename('탑승건수')
alight_daejang

In [ ]:
alight_daejang.plot(kind='bar')

In [ ]:
## 4.3. 목적지 분포 확인

In [ ]:
# 하루 전체 분포 - 대장동에서 승차한 통행 중 하차 정류장들의 위치 분포 확인
board_daejang = linked_daejang[linked_daejang.정산사승차정류장ID.isin(daejang_stations)]
board_daejang_no_card = board_daejang.drop(columns=['가상카드번호', '트랜잭션ID'])
board_daejang_no_card.to_csv('대장동_하차분포_카드제외.csv')

In [ ]:
board_daejang.

In [ ]:
### 4.3.1. 승차 도착지점과 출발지점이 같을까?

In [ ]:
# 500m 근거리에 있는가?
# 산출방식: 가상카드번호로 묶었을 때 발생한 목적 통행이 2개인 경우만 분석 가능 -> 첫번째 행의 하차정류장과 두번째 행의 승차정류장의 위치가 500m이하인지 유무 파악


# 승차, 하차 정류장 위치 붙이기
round_trip_daejang['board_geometry'] = gpd.points_from_xy(round_trip_daejang['승차정류장X좌표'],round_trip_daejang['승차정류장Y좌표'])
round_trip_daejang['alight_geometry'] = gpd.points_from_xy(round_trip_daejang['하차정류장X좌표'],round_trip_daejang['하차정류장Y좌표'])

# 좌표계 5179로 변환
board = gpd.GeoSeries(round_trip_daejang['board_geometry'])
alight = gpd.GeoSeries(round_trip_daejang['alight_geometry'])
board.crs = 'EPSG:4326'
alight.crs = 'EPSG:4326'
round_trip_daejang['board_geometry'] = board.to_crs('EPSG:5179')
round_trip_daejang['alight_geometry'] = alight.to_crs('EPSG:5179')

def workplace_distance(df, limit=1000):
    first_geometry = df.iloc[0]['alight_geometry'] # 첫번째 통행의 하차 지점
    second_geometry = df.iloc[1]['board_geometry'] # 두번째 통행의 승차 지점
    distance = first_geometry.distance(second_geometry)
    # # 거리 필터
    # if distance <= limit:
    #     return distance
    # else:
    #     return False
    return round(distance,2)

limit_distance = round_trip_daejang.groupby('가상카드번호').apply(workplace_distance).to_frame().reset_index().rename(columns={0:'distance'})
limit_distance.distance.value_counts()
# limit_true[~limit_true['is_round']]
limit_distance

In [ ]:
# 구간별로 나누기
bins = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000,2000, 5000, 1000000]
labels = ['0~100', '100~200', '200~300', '300~400', '400~500', '500~600', '600~700', '700~800', '800~900', '900~1000', '1000~2000','2000~5000',  '5000~']
distances = pd.cut(limit_distance['distance'], bins=bins, labels=labels, right=False)
distances.value_counts()

In [ ]:
round_trip_daejang[round_trip_daejang['가상카드번호'] =='++ieWI9jeC16qOPMM6LWVWl1ubiRvEZp0YSr5HHjfPg=']
# round_trip_daejang

In [ ]:
## 4.4. 첫통행 승차와 두번째 하차 시간 패턴 분석

In [ ]:
# 왕복통행 중 대장동 출발&도착인 건만 추출
# 출발한 통행들 집계
round_trip_daejang[round_trip_daejang.정산사승차정류장ID

In [ ]:
agg_way = {'첫통행하차일시':('하차일시','min'), '두번째통행승차일시':('승차일시','max')}
time_collection = round_trip_daejang.groupby('가상카드번호').agg(**agg_way).reset_index()
len(time_collection[time_collection['첫통행하차일시']>time_collection['두번째통행승차일시']]) # 집계방식 이상 점검 -> none
time_collection

In [ ]:
heatmap_data

In [ ]:
heatmap_data.sum()

In [ ]:
# 왕복통행 1631건 중
time_collection['1st_arrival'] = time_collection['첫통행하차일시'].dt.floor('H')
time_collection['2nd_departure'] = time_collection['두번째통행승차일시'].dt.floor('H')

# 교차 테이블 생성
heatmap_data = time_collection.groupby(['1st_arrival', '2nd_departure']).size().unstack(fill_value=0)

# 히트맵 시각화
plt.figure(figsize=(10,6))
sns.heatmap(heatmap_data, annot=True, fmt='d', cmap ='coolwarm')
plt.title('첫통행하차일시, 두번째통행승차일시 조합')
plt.xlabel('두번째통행승차일시')
plt.ylabel('첫통행하차일시')
plt.show() # 7-10 /  17-20

In [ ]:
# 5. 통근 인원 패턴 분석

In [ ]:
first_card

In [ ]:
first_card = first_trips[first_trips.정산사승차정류장ID.isin(daejang_stations)]

In [ ]:
first_card.가상카드번호.nunique()

In [ ]:
# 방문인구 필터( 들어왔다 나가는 인구)
first_trips = round_trip_daejang.groupby('가상카드번호').nth(0) # 1631개
second_trips = round_trip_daejang.groupby('가상카드번호').nth(1) # 1631개

# 첫번째 통행의 출발지가 대장동 & 두번째 통행의 도착지가 대장동인 곳
first_card = first_trips[first_trips.정산사승차정류장ID.isin(daejang_stations)] # 1357개
first_card = first_card.가상카드번호.unique()
second_card = second_trips[second_trips.정산사하차정류장ID.isin(daejang_stations)] # 1357개
second_card = second_card.가상카드번호.unique()
valid_id = set(first_card) & set(second_card)
len(valid_id) # 1353개 통행 정상 통행

round_trip_daejang = round_trip_daejang[round_trip_daejang.가상카드번호.isin(valid_id)]
round_trip_daejang # 3262 -> 2706개

In [ ]:
# 퍼센트 정리

# 왕복통행 - 1631명(총 목적통행 횟수-3262건)
# 이 중 대장동에서 출발해서 돌아오는 통행수
# - 1353명(총 목적통행 수 - 2706건)
# 그 중 7-10시, 17-20시 적용 시 직장통행 패턴 적용시
# - 430명(총 목적통행 수 - 860건)

In [ ]:
# 몇 퍼센트가 구분되는지 체크
# 시간대 설정해서 필터 시 몇 퍼센트가 되는지
# 첫출발하차시간 7-10 /  두번째 출발 시간 17-20 적용

morning_filter = first_trips[(pd.Timestamp('2024-01-02-07:00:00')<=first_trips.하차일시)&(first_trips.하차일시<=pd.Timestamp('2024-01-02-10:00:00'))] # 1353-> 732건
night_filter = second_trips[(pd.Timestamp('2024-01-02-17:00:00')<=second_trips.승차일시)&(second_trips.승차일시<=pd.Timestamp('2024-01-02-20:00:00'))] # 612건

morning_card = morning_filter.가상카드번호.unique()
night_card = night_filter.가상카드번호.unique()
valid_id = set(morning_card) & set(night_card)
len(valid_id) # 둘다 만족하는 케이스 430명
commuting_trip = round_trip_daejang[round_trip_daejang.가상카드번호.isin(valid_id)]
commuting_trip # 860건

In [ ]:
# 6. 시계열 패턴 분석